In [5]:
import random
import logging
import numpy as np
import torch
import csv
from pytorchfi import core
from finetune import Ground_truth_model
from errors import stuck_at_one, stuck_at_zero, bit_flip
from bitflips import *
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification


In [10]:
class Perturbed_model:
    def __init__(self,model_,tokenizer,batch_size,input_shape,layer_types,use_cuda=False):
        self.ground_truth=model_
        self.inj_model=core.fault_injection(self.ground_truth.model,batch_size,input_shape,layer_types,use_cuda)
        self.tokenizer=tokenizer
    def setup(self):
        self.inj_model._traverse_model_set_hooks()
        print(self.inj_model.print_pytorchfi_layer_summary)
    def experiment(self,test_loader,n=100):
        accuracies=[]
        for i in range(n):
            # Inject faults into the model
            # Here, we use random neuron perturbation as an example
            # You can customize the fault injection parameters as needed
            random_inj_per_layer_batched(self.inj_model)
                
            # Evaluate the faulty model
            accuracy = self.evaluate(self.inj_model, test_loader)
            accuracies.append(accuracy)
                
                # Reset the fault injection for the next iteration
            self.inj_model.fi_reset()

        # Visualize the accuracy results
        plt.plot(range(n), accuracies, marker='o')
        plt.xlabel('Iteration')
        plt.ylabel('Accuracy')
        plt.title('Model Accuracy Under Fault Injections')
        plt.show()
    def evaluate(self, dataloader):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        self.model.eval()

        total, correct = 0, 0
        with torch.no_grad():
            for batch in dataloader:
                input_ids, attention_mask, labels = (t.to(device) for t in batch)
                outputs = self.model(input_ids, attention_mask=attention_mask)
                _, preds = torch.max(outputs.logits, dim=1)
                total += labels.size(0)
                correct += (preds == labels).sum().item()

        accuracy = 100 * correct / total
        return accuracy


In [11]:
class IMDbDataset(Dataset):
    def __init__(self, tokenizer, split="train", max_length=128):
        self.dataset = load_dataset("imdb", split=split)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset[idx]["text"]
        label = self.dataset[idx]["label"]
        inputs = self.tokenizer(
            text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt"
        )
        return inputs["input_ids"].squeeze(0), inputs["attention_mask"].squeeze(0), torch.tensor(label)


In [12]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

ground_truth = Ground_truth_model(model,tokenizer)

train_dataset = IMDbDataset(ground_truth.tokenizer, split="train[:5000]")  # Use a subset for speed
test_dataset = IMDbDataset(ground_truth.tokenizer, split="test[:1000]")    # Use a subset for evaluation


train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

ground_truth.train(train_dataloader,epochs=3)
accuracy=ground_truth.evaluate(test_dataloader)
print(f"Initial accuracy on IMDb test set: {accuracy:.2f}%")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [ ]:
perturb_model = Perturbed_model(ground_truth.model,ground_truth.tokenizer,batch_size=16, input_shape=(3,224,224),layer_types=["attn"])
perturb_model.experiment(test_dataloader)